## Grocery calculator

In [1]:
import pandas as pd
import requests
import json
import regex as re
import numpy as np
import os
from dotenv import load_dotenv

### Setup API

In [2]:
load_dotenv()

True

In [3]:
url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers = {'Content-type': 'application/json'}
api_key = os.getenv('PROJECT_API_KEY')


In [4]:
print(f"API Key loaded: {'✓' if api_key else '✗'}")

API Key loaded: ✓


### Load series data

In [5]:
df = pd.read_csv('data/series-data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   series_id    25 non-null     object
 1   item_name    25 non-null     object
 2   description  25 non-null     object
 3   unit         25 non-null     object
 4   category     25 non-null     object
 5   url          25 non-null     object
dtypes: object(6)
memory usage: 1.3+ KB


In [6]:
series_id = df['series_id']

In [7]:
data = {
    "seriesid": series_id.tolist(), 
    "startyear": "2024",
    "endyear": "2025",
    "registrationkey": api_key
}

In [8]:
response = requests.post(url, data=json.dumps(data), headers=headers)
response_data = response.json()

In [9]:
response_data

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 186,
 'message': [],
 'Results': {'series': [{'seriesID': 'APU0000701312',
    'data': [{'year': '2025',
      'period': 'M11',
      'periodName': 'November',
      'latest': 'true',
      'value': '1.065',
      'footnotes': [{}]},
     {'year': '2025',
      'period': 'M10',
      'periodName': 'October',
      'value': '-',
      'footnotes': [{'code': 'X',
        'text': 'Data unavailable due to the 2025 lapse in appropriations'}]},
     {'year': '2025',
      'period': 'M09',
      'periodName': 'September',
      'value': '1.060',
      'footnotes': [{}]},
     {'year': '2025',
      'period': 'M08',
      'periodName': 'August',
      'value': '1.059',
      'footnotes': [{}]},
     {'year': '2025',
      'period': 'M07',
      'periodName': 'July',
      'value': '1.068',
      'footnotes': [{}]},
     {'year': '2025',
      'period': 'M06',
      'periodName': 'June',
      'value': '1.061',
      'footnotes': [{}]},
     {'ye

In [10]:
response_df = pd.DataFrame(response_data['Results']['series'])

In [11]:
exploded_df = pd.json_normalize(response_data['Results']['series'], record_path='data')

In [12]:
exploded_df = response_df.explode('data')
exploded_df = pd.concat([exploded_df.drop(columns=['data']), exploded_df['data'].apply(pd.Series)], axis=1)
exploded_df

,seriesID,year,period,periodName,latest,value,footnotes
0,APU0000701312,2025,M11,November,true,1.065,[{}]
0,APU0000701312,2025,M10,October,NaN,-,"[{'code': 'X', 'text': 'Data unavailable due t..."
0,APU0000701312,2025,M09,September,NaN,1.060,[{}]
0,APU0000701312,2025,M08,August,NaN,1.059,[{}]
0,APU0000701312,2025,M07,July,NaN,1.068,[{}]
...,...,...,...,...,...,...,...
24,APU0000FF1101,2024,M05,May,NaN,4.118,[{}]
24,APU0000FF1101,2024,M04,April,NaN,4.061,[{}]
24,APU0000FF1101,2024,M03,March,NaN,4.106,[{}]
24,APU0000FF1101,2024,M02,February,NaN,4.105,[{}]


In [16]:
exploded_df.to_csv('bls_data_nov_2025.csv', index=False)

In [19]:
df2 = exploded_df.merge(df[['series_id', 'item_name']], left_on='seriesID', right_on='series_id', how='left')


In [21]:
df2.drop(columns=['series_id'], inplace=True)

In [20]:
// df2 = exploded_df.merge(df[['series_id', 'item_name']], left_on='seriesID', right_on='series_id', how='left')
df2.drop(columns=['series_id'], inplace=True)
// df2

SyntaxError: invalid syntax (3420739958.py, line 1)

In [22]:
df2

,seriesID,year,period,periodName,latest,value,footnotes,item_name
0,APU0000701312,2025,M11,November,true,1.065,[{}],Rice
1,APU0000701312,2025,M10,October,NaN,-,"[{'code': 'X', 'text': 'Data unavailable due t...",Rice
2,APU0000701312,2025,M09,September,NaN,1.060,[{}],Rice
3,APU0000701312,2025,M08,August,NaN,1.059,[{}],Rice
4,APU0000701312,2025,M07,July,NaN,1.068,[{}],Rice
...,...,...,...,...,...,...,...,...
568,APU0000FF1101,2024,M05,May,NaN,4.118,[{}],Chicken breast
569,APU0000FF1101,2024,M04,April,NaN,4.061,[{}],Chicken breast
570,APU0000FF1101,2024,M03,March,NaN,4.106,[{}],Chicken breast
571,APU0000FF1101,2024,M02,February,NaN,4.105,[{}],Chicken breast


In [13]:
df2.to_csv('data/raw-data.csv', index=False)

NameError: name 'df2' is not defined

In [23]:
df2['date'] = df2['periodName'] + ' ' + df2['year']

In [24]:
df2

,seriesID,year,period,periodName,latest,value,footnotes,item_name,date
0,APU0000701312,2025,M11,November,true,1.065,[{}],Rice,November 2025
1,APU0000701312,2025,M10,October,NaN,-,"[{'code': 'X', 'text': 'Data unavailable due t...",Rice,October 2025
2,APU0000701312,2025,M09,September,NaN,1.060,[{}],Rice,September 2025
3,APU0000701312,2025,M08,August,NaN,1.059,[{}],Rice,August 2025
4,APU0000701312,2025,M07,July,NaN,1.068,[{}],Rice,July 2025
...,...,...,...,...,...,...,...,...,...
568,APU0000FF1101,2024,M05,May,NaN,4.118,[{}],Chicken breast,May 2024
569,APU0000FF1101,2024,M04,April,NaN,4.061,[{}],Chicken breast,April 2024
570,APU0000FF1101,2024,M03,March,NaN,4.106,[{}],Chicken breast,March 2024
571,APU0000FF1101,2024,M02,February,NaN,4.105,[{}],Chicken breast,February 2024


In [17]:
df2['value'] = df2['value'].astype(float).apply(lambda x: round(x, 2))
df2

,seriesID,year,period,periodName,latest,value,footnotes,item_name,date
0,APU0000701312,2025,M08,August,true,1.06,[{}],Rice,August 2025
1,APU0000701312,2025,M07,July,NaN,1.07,[{}],Rice,July 2025
2,APU0000701312,2025,M06,June,NaN,1.06,[{}],Rice,June 2025
3,APU0000701312,2025,M05,May,NaN,1.07,[{}],Rice,May 2025
4,APU0000701312,2025,M04,April,NaN,1.03,[{}],Rice,April 2025
...,...,...,...,...,...,...,...,...,...
495,APU0000FF1101,2024,M05,May,NaN,4.12,[{}],Chicken breast,May 2024
496,APU0000FF1101,2024,M04,April,NaN,4.06,[{}],Chicken breast,April 2024
497,APU0000FF1101,2024,M03,March,NaN,4.11,[{}],Chicken breast,March 2024
498,APU0000FF1101,2024,M02,February,NaN,4.11,[{}],Chicken breast,February 2024


In [25]:
sorted_columns = sorted(df2['date'].unique(), key=lambda x: pd.to_datetime(x))
pivot_sorted = df2.pivot(index='seriesID', columns='date', values='value')[sorted_columns]
pivot_sorted

date,January 2024,February 2024,March 2024,April 2024,May 2024,June 2024,July 2024,August 2024,September 2024,October 2024,...,February 2025,March 2025,April 2025,May 2025,June 2025,July 2025,August 2025,September 2025,October 2025,November 2025
seriesID,,,,,,,,,,,,,,,,,,,,,
APU0000701312,1.000,1.001,1.014,1.025,1.002,1.018,1.039,1.085,1.078,1.056,...,1.030,1.038,1.031,1.067,1.061,1.068,1.059,1.060,-,1.065
APU0000701322,1.425,1.432,1.420,1.422,1.377,1.435,1.432,1.421,1.431,1.445,...,1.381,1.379,1.361,1.322,1.318,1.306,1.294,1.205,-,1.300
APU0000702111,2.033,2.006,1.997,1.998,1.971,1.973,1.975,1.951,1.976,1.937,...,1.925,1.880,1.913,1.876,1.864,1.851,1.841,1.867,-,1.790
APU0000702421,5.019,5.248,5.145,5.121,5.124,5.017,5.102,5.046,5.173,4.852,...,4.658,4.863,4.956,5.049,5.108,5.264,5.122,5.150,-,5.331
APU0000703113,6.778,6.651,6.734,6.809,6.847,6.893,7.169,6.918,7.223,7.220,...,7.244,7.476,7.551,7.688,7.672,8.035,7.950,8.182,-,8.232
APU0000703511,8.106,7.883,8.031,8.110,8.247,7.927,8.059,8.226,8.148,8.154,...,8.485,8.554,8.627,8.401,8.457,8.690,9.076,9.360,-,9.665
APU0000704111,6.614,6.560,6.609,6.637,6.818,6.827,6.883,6.785,6.955,6.869,...,6.798,6.978,7.009,6.985,7.098,7.118,7.208,7.290,-,6.928
APU0000704312,5.532,5.666,5.572,5.594,5.668,5.567,5.727,5.633,5.630,5.670,...,5.520,5.603,5.473,5.453,5.379,5.685,5.432,5.478,-,5.462
APU0000708111,2.522,2.996,2.992,2.864,2.699,2.715,3.080,3.204,3.821,3.370,...,5.897,6.227,5.122,4.548,3.775,3.599,3.587,3.488,-,2.860


In [26]:
df3 = pivot_sorted.merge(df, left_on='seriesID', right_on='series_id', how='left')

In [27]:
column_order = ['series_id', 'item_name', 'description', 'unit', 'category', 'url'] + [col for col in df3.columns if col not in ['series_id', 'item_name', 'description', 'unit', 'category', 'url']]
df4 = df3[column_order]
df4

,series_id,item_name,description,unit,category,url,January 2024,February 2024,March 2024,April 2024,...,February 2025,March 2025,April 2025,May 2025,June 2025,July 2025,August 2025,September 2025,October 2025,November 2025
0,APU0000701312,Rice,"Rice, white, long grain, uncooked, per lb.",per lb.,Pantry,https://fred.stlouisfed.org/graph/fredgraph.cs...,1.000,1.001,1.014,1.025,...,1.030,1.038,1.031,1.067,1.061,1.068,1.059,1.060,-,1.065
1,APU0000701322,Pasta,"Spaghetti and macaroni, per lb.",per lb.,Pantry,https://fred.stlouisfed.org/graph/fredgraph.cs...,1.425,1.432,1.420,1.422,...,1.381,1.379,1.361,1.322,1.318,1.306,1.294,1.205,-,1.300
2,APU0000702111,White bread,"Bread, white, pan, per lb.",per lb.,Pantry,https://fred.stlouisfed.org/graph/fredgraph.cs...,2.033,2.006,1.997,1.998,...,1.925,1.880,1.913,1.876,1.864,1.851,1.841,1.867,-,1.790
3,APU0000702421,Cookies,"Cookies, chocolate chip, per lb.",per lb.,Snacks,https://fred.stlouisfed.org/graph/fredgraph.cs...,5.019,5.248,5.145,5.121,...,4.658,4.863,4.956,5.049,5.108,5.264,5.122,5.150,-,5.331
4,APU0000703113,Ground beef,"Ground beef, lean and extra lean, per lb.",per lb.,Meat,https://fred.stlouisfed.org/graph/fredgraph.cs...,6.778,6.651,6.734,6.809,...,7.244,7.476,7.551,7.688,7.672,8.035,7.950,8.182,-,8.232
5,APU0000703511,Steak,"Steak, round, USDA Choice, boneless, per lb.",per lb.,Meat,https://fred.stlouisfed.org/graph/fredgraph.cs...,8.106,7.883,8.031,8.110,...,8.485,8.554,8.627,8.401,8.457,8.690,9.076,9.360,-,9.665
6,APU0000704111,Bacon,"Bacon, sliced, per lb.",per lb.,Meat,https://fred.stlouisfed.org/graph/fredgraph.cs...,6.614,6.560,6.609,6.637,...,6.798,6.978,7.009,6.985,7.098,7.118,7.208,7.290,-,6.928
7,APU0000704312,Ham,"Ham, boneless, excluding canned, per lb.",per lb.,Meat,https://fred.stlouisfed.org/graph/fredgraph.cs...,5.532,5.666,5.572,5.594,...,5.520,5.603,5.473,5.453,5.379,5.685,5.432,5.478,-,5.462
8,APU0000708111,Eggs,"Eggs, grade A, large, per doz.",per doz.,Dairy,https://fred.stlouisfed.org/graph/fredgraph.cs...,2.522,2.996,2.992,2.864,...,5.897,6.227,5.122,4.548,3.775,3.599,3.587,3.488,-,2.860
9,APU0000709112,Milk,"Milk, fresh, whole, fortified, per gal.",per gal.,Dairy,https://fred.stlouisfed.org/graph/fredgraph.cs...,3.958,3.940,3.893,3.868,...,4.026,4.050,4.074,4.022,4.029,4.162,4.171,4.129,-,3.997


In [28]:
df4

,series_id,item_name,description,unit,category,url,January 2024,February 2024,March 2024,April 2024,...,February 2025,March 2025,April 2025,May 2025,June 2025,July 2025,August 2025,September 2025,October 2025,November 2025
0,APU0000701312,Rice,"Rice, white, long grain, uncooked, per lb.",per lb.,Pantry,https://fred.stlouisfed.org/graph/fredgraph.cs...,1.000,1.001,1.014,1.025,...,1.030,1.038,1.031,1.067,1.061,1.068,1.059,1.060,-,1.065
1,APU0000701322,Pasta,"Spaghetti and macaroni, per lb.",per lb.,Pantry,https://fred.stlouisfed.org/graph/fredgraph.cs...,1.425,1.432,1.420,1.422,...,1.381,1.379,1.361,1.322,1.318,1.306,1.294,1.205,-,1.300
2,APU0000702111,White bread,"Bread, white, pan, per lb.",per lb.,Pantry,https://fred.stlouisfed.org/graph/fredgraph.cs...,2.033,2.006,1.997,1.998,...,1.925,1.880,1.913,1.876,1.864,1.851,1.841,1.867,-,1.790
3,APU0000702421,Cookies,"Cookies, chocolate chip, per lb.",per lb.,Snacks,https://fred.stlouisfed.org/graph/fredgraph.cs...,5.019,5.248,5.145,5.121,...,4.658,4.863,4.956,5.049,5.108,5.264,5.122,5.150,-,5.331
4,APU0000703113,Ground beef,"Ground beef, lean and extra lean, per lb.",per lb.,Meat,https://fred.stlouisfed.org/graph/fredgraph.cs...,6.778,6.651,6.734,6.809,...,7.244,7.476,7.551,7.688,7.672,8.035,7.950,8.182,-,8.232
5,APU0000703511,Steak,"Steak, round, USDA Choice, boneless, per lb.",per lb.,Meat,https://fred.stlouisfed.org/graph/fredgraph.cs...,8.106,7.883,8.031,8.110,...,8.485,8.554,8.627,8.401,8.457,8.690,9.076,9.360,-,9.665
6,APU0000704111,Bacon,"Bacon, sliced, per lb.",per lb.,Meat,https://fred.stlouisfed.org/graph/fredgraph.cs...,6.614,6.560,6.609,6.637,...,6.798,6.978,7.009,6.985,7.098,7.118,7.208,7.290,-,6.928
7,APU0000704312,Ham,"Ham, boneless, excluding canned, per lb.",per lb.,Meat,https://fred.stlouisfed.org/graph/fredgraph.cs...,5.532,5.666,5.572,5.594,...,5.520,5.603,5.473,5.453,5.379,5.685,5.432,5.478,-,5.462
8,APU0000708111,Eggs,"Eggs, grade A, large, per doz.",per doz.,Dairy,https://fred.stlouisfed.org/graph/fredgraph.cs...,2.522,2.996,2.992,2.864,...,5.897,6.227,5.122,4.548,3.775,3.599,3.587,3.488,-,2.860
9,APU0000709112,Milk,"Milk, fresh, whole, fortified, per gal.",per gal.,Dairy,https://fred.stlouisfed.org/graph/fredgraph.cs...,3.958,3.940,3.893,3.868,...,4.026,4.050,4.074,4.022,4.029,4.162,4.171,4.129,-,3.997


In [29]:
df4.to_csv('data/clean_data.csv', index=False)